# Downloading Historical Price Data Using the OpenBB SDK

This notebook demonstrates the different ways to approach loading historical price data using the OpenBB SDK.  There are obvious methods, and some less conventional ones. To begin, let's look at the Stocks module, but first, we need to initialize the notebook with the import statements block.

## Import Statements

In [2]:
import pandas as pd
from openbb_terminal.sdk import openbb

## The Stocks Module

The `load` function is likely the first place to reach when requesting daily, or intraday, historical price data.  `openbb.stocks.load()` is a versatile data aggregator, and not only for stocks.

### openbb.stocks.load()

The input parameters for the function:

```python
Parameters
----------
symbol: str
    Ticker to get data
start_date: str or datetime, optional
    Start date to get data from with. - datetime or string format (YYYY-MM-DD)
interval: int
    Interval (in minutes) to get data 1, 5, 15, 30, 60 or 1440
end_date: str or datetime, optional
    End date to get data from with. - datetime or string format (YYYY-MM-DD)
prepost: bool
    Pre and After hours data
source: str
    Source of data extracted
weekly: bool
    Flag to get weekly data
monthly: bool
    Flag to get monthly data
verbose: bool
    Display verbose information on what was the symbol that was loaded
```


Set `verbose = False` to silence the console print.  Execute the two cells below to demonstrate.

In [2]:
df_daily = openbb.stocks.load(symbol = 'spy')

Loading Daily data for SPY with starting period 2020-05-18.

In [3]:
df_daily = openbb.stocks.load(symbol = 'spy', verbose = False)

To load the entire history available from a source, pick a starting date well beyond what it might be. For example, `1900-01-01`

In [4]:
df_daily = openbb.stocks.load(symbol = 'spy', start_date = '1990-01-01')

Loading Daily data for SPY with starting period 1993-01-29.

Weekly and monthly intervals are selected with either argument as `True`.  The `load` function can be used as an input parameter to functions, like `candle`.  This draws a chart without needing to load anything into memory.

In [14]:
openbb.stocks.candle(
    data = openbb.stocks.load(symbol = 'spy', start_date = '2020-06-01', weekly = True, verbose = False),
    symbol = "SPY - Weekly"
)

In [13]:
openbb.stocks.candle(
    data = openbb.stocks.load(symbol = 'spy', start_date = '2013-06-01', monthly = True, verbose = False),
    symbol = "SPY Monthly",
    ha = True,
)

The default settings retrieve daily, intraday data is selected using the `interval` argument, with a choice of: `[1, 5, 15, 30, 60]`.  Include pre/post market data by setting the argument, `prepost = True`.  The amount of history for each interval, and level of market coverage, will vary by the source, which is selectable via the `source` argument.  The block below packs an object with all intervals.

In [121]:
from datetime import datetime,timedelta

def load_historical(symbol: str = "", start_date: str = "", end_date: str = "", source: str = "", prepost: bool = False) -> object:

    if symbol == "":
        print("Please enter a ticker symbol")
    if start_date == "":
        start_date = (datetime.now() - timedelta(weeks = 104)).date()
    if end_date == "":
        end_date = datetime.now().date()
    if source == "":
        source = "YahooFinance"

    class HistoricalPrices:
        def __init__(self) -> None:
            self.one: pd.DataFrame = openbb.stocks.load(
                symbol=symbol, start_date=start_date, end_date=end_date, interval = 1, source=source, verbose = False, prepost = prepost
            ).convert_dtypes()
            self.five: pd.DataFrame = openbb.stocks.load(
                symbol=symbol, start_date=start_date, end_date=end_date, interval = 5, source = source, verbose = False, prepost = prepost
            ).convert_dtypes()
            self.fifteen: pd.DataFrame = openbb.stocks.load(
                symbol=symbol, start_date=start_date, end_date=end_date, interval = 15, verbose = False, prepost = prepost
            ).convert_dtypes()
            self.thirty: pd.DataFrame = openbb.stocks.load(
                symbol=symbol, start_date=start_date, end_date=end_date, interval = 30, verbose = False, prepost = prepost
            ).convert_dtypes()
            self.sixty: pd.DataFrame = openbb.stocks.load(
                symbol=symbol, start_date=start_date, end_date=end_date, interval = 60, verbose = False, prepost = prepost
            ).convert_dtypes()
            self.daily: pd.DataFrame = openbb.stocks.load(
                symbol=symbol, start_date=start_date, end_date=end_date, source = source, verbose = False
            ).convert_dtypes()
            self.weekly: pd.DataFrame = openbb.stocks.load(
                symbol=symbol, start_date=start_date, end_date=end_date, source = source, weekly = True, verbose = False
            ).convert_dtypes()
            self.monthly: pd.DataFrame = openbb.stocks.load(
                symbol=symbol, start_date=start_date, end_date=end_date, source=source, monthly = True, verbose = False
            ).convert_dtypes()
    prices = HistoricalPrices()

    return prices

In [122]:
prices = load_historical("SPY")
openbb.stocks.candle(data = prices.fifteen["2023-05-15":], symbol = "SPY - Fifteen Minute Candles")

To demonstrate the difference between sources, let's compare values for daily volume from several sources.

In [113]:
# Yahoo was the source for the previously loaded data.
yahoo = prices

polygon = load_historical("SPY", source = "Polygon")
alphavantage = load_historical("SPY", source = "AlphaVantage")

In [114]:
# Make a new DataFrame with just the volume columns
compare = pd.DataFrame()
compare["AlphaVantage Volume"] = alphavantage.daily["Volume"]
compare["Polygon Volume"] = polygon.daily["Volume"]
compare["Yahoo Volume"] = yahoo.daily["Volume"]

compare.tail(2)

,AlphaVantage Volume,Polygon Volume,Yahoo Volume
date,,,
2023-05-22,60745399,60754935,60745400
2023-05-23,85145553,86383531,86301900


Then at the one-minute interval.

In [115]:
compare_intraday = pd.DataFrame()
compare_intraday["AlphaVantage Volume"] = alphavantage.one["Volume"]
compare_intraday["Polygon Volume"] = polygon.one["Volume"]
compare_intraday["Yahoo Volume"] = yahoo.one["Volume"]
compare_intraday.tail(10)

,AlphaVantage Volume,Polygon Volume,Yahoo Volume
date,,,
2023-05-23 15:51:00,485786,485786,485808
2023-05-23 15:52:00,644560,644560,644023
2023-05-23 15:53:00,368111,368111,368608
2023-05-23 15:54:00,684340,684340,684180
2023-05-23 15:55:00,773708,773708,773810
2023-05-23 15:56:00,476353,476353,476358
2023-05-23 15:57:00,694285,694285,694322
2023-05-23 15:58:00,1009322,1009322,1009331
2023-05-23 15:59:00,2831803,2831803,2814414


## Other Types of Symbols

Other types of assets and ticker symbols can be loaded from `openbb.stocks.load()`, below are some examples but not an exhaustive list.

### Share Classes

Some sources use `-` as the distinction between a share class, e.g., `BRK-A` and `BRK-B`. Other formats include:

- A period: `BRK.A`
- A slash: `BRK/A`
- No separator, the share class becomes the fourth or fifth letter.

Some may even use a combination, or accept multiple variations.  Sometimes there is no real logic behind the additional characters, `GOOGL` vs. `GOOG`.  These are known unknown variables of ticker symbology, what's good for one source may return errors from another. 

### Indexes

Sources will have their own treatment of these symbols, some examples are:

- YahooFinance: ^IXIC
- FRED (not the `load` function): `openbb.economy.fred(["NASDAQCOM"])`
- Polygon: I:NDX

### Currencies

FX symbols face the same dilemna as share classes, there are several variations of the same symbol.

- YahooFinance: `EURUSD=X`
- Polygon: `C:EURUSD`
- AlphaVantage: `EURUSD`

### Crypto

Similar, but different to FX tickers.

- YahooFinance: `BTC-USD`
- Polygon: `X:BTCUSD`
- AlphaVantage: `BTCUSD`

### Futures

Historical prices for active contracts, and the continuous front-month contracts, are able to be loaded via `YahooFinance`.

- Continuous front-month: `CL=F`
- December 2023 contract: `CLZ23.NYM`
- March 2024 contract: `CLH23.NYM`

Individual contracts will require knowing which of the CME venues the future is listed on. `["NYM", "NYB", "CME", "CBT"]`.  Use, `openbb.futures.search()`, to determine which.

### Options

Individual options contracts are also loadable from `openbb.stocks.load()`.

- YahooFinance: `SPY230616P00400000`
- Polygon: `O:SPY230616P00400000`

What the `load` function **cannot** do is batch download.  For this type of functionality, we can go elsewhere in the SDK.

### openbb.stocks.ca.hist()

This function, within the Comparison Analysis sub-module, offers the ability to return data for multiple tickers as a single Pandas DataFrame, and targeting a single data point.  Choices are: open, high, low, close, adjust close, volume, and return.  It is chosen by using the optional argument, `candle_type`, with the first letter of the candle type as the value - `a` for Adjusted Close.  The list of tickers must be encapsulated with square brackets, [], and quotation marks, `""`, for each individual symbol.

In [126]:
tickers = ["META", "AAPL", "AMZN", "NFLX", "GOOG", "MSFT", "TSLA"]
df_compare = openbb.stocks.ca.hist(similar = tickers, start_date = "2023-01-01", candle_type = 'a')

df_compare.tail(3)

,META,AAPL,AMZN,NFLX,GOOG,MSFT,TSLA
Date,,,,,,,
2023-05-19,245.639999,175.160004,116.250000,365.359985,123.250000,318.339996,180.139999
2023-05-22,248.320007,174.199997,115.010002,363.010010,125.870003,321.179993,188.869995
2023-05-23,246.740005,171.559998,114.989998,355.989990,123.290001,315.260010,185.770004


## The Economy Module

It may not be an obvious choice, but it can get the job done in a similar way as `openbb.stocks.ca.hist()`.

### openbb.economy.index()

This function can be coaxed into completing the same task, with slight differences being the `column` argument, and `returns` is a boolean choice for calculating the cumulative returns over the entire history.

In [25]:
df_compare2 = openbb.economy.index(indices = tickers, column = 'Adj Close')

df_compare2.tail(3)

,META,AAPL,AMZN,NFLX,GOOG,MSFT,TSLA
Date,,,,,,,
2023-05-18,246.850006,175.050003,118.150002,371.290009,123.519997,318.519989,176.889999
2023-05-19,245.639999,175.160004,116.250000,365.359985,123.250000,318.339996,180.139999
2023-05-22,251.149994,173.779999,114.544998,366.720001,126.144997,321.709991,184.540100


Some daily historical prices, or index levels, are part of the FRED library.  For example, the Nasdaq Composite Index.

In [128]:
sp500 = openbb.economy.fred(["SP500"])

# Note that the return from FRED is a Tuple, and the second element is the Series ID and description.

print(sp500[1])

sp500[0].tail(1)

{'SP500': {'title': 'S&P 500', 'units': 'Index'}}


,SP500
2023-05-23,4145.58


This endpoint also has a fast method for plotting series, simply add `_chart` to the command.

In [131]:
openbb.economy.fred_chart(["DJCA", "NASDAQCOM"])

## The Crypto Module

Similarly, the `crypto` module has its own nuances.

### openbb.crypto.load()

This version of the `load` function has three sources: `YahooFinance`, `CoinGecko`, and `CCXT`. Selecting `CCXT` will activate the `--exchange` argument, which allows the user to target a specific venue.  

Choices of exchange are: [

    ace,alpaca,ascendex,bequant,bigone,binance,binancecoinm,binanceus,binanceusdm,bit2c,bitbank,bitbay,bitcoincom,
    bitfinex,bitfinex2,bitflyer,bitforex,bitget,bithumb,bitmart,bitmex,bitopro,bitpanda,bitrue,bitso,bitstamp,bitstamp1,
    bittrex,bitvavo,bkex,bl3p,btcalpha,btcbox,btcex,btcmarkets,btctradeua,btcturk,buda,bybit,cex,coinbase,coinbaseprime,coinbasepro,
    coincheck,coinex,coinfalcon,coinmate,coinone,coinspot,cryptocom,currencycom,delta,deribit,digifinex,exmo,flowbtc,fmfwio,gate,
    gateio,gemini,hitbtc,hitbtc3,hollaex,huobi,huobijp,huobipro,idex,independentreserve,indodax,itbit,kraken,krakenfutures,kucoin,
    kucoinfutures,kuna,latoken,lbank,lbank2,lykke,mercado,mexc,mexc3,ndax,novadax,oceanex,okcoin,okex,okex5,okx,paymium,phemex,
    poloniex,poloniexfutures,probit,ripio,stex,tidex,timex,tokocrypto,upbit,wavesexchange,wazirx,whitebit,woo,yobit,zaif,zb,zonda
]

**Note**: Asset coverage and history varies widely by exchange.

In [149]:
btc_usdt = openbb.crypto.load(symbol = 'BTC', to_symbol = 'USDT', source = 'CCXT', exchange = 'bitcoincom', start_date = '2010-01-01')
openbb.qa.line(data = btc_usdt["Close"], title = " Daily BTC/USDT", log_y = True)

### openbb.crypto.dd.mt()

This function returns crpyto time-series data from Messari, selectable by `timeseries_id`. Find the appropriate value with, `openbb.crypto.dd.get_mt()`. 

In [32]:
openbb.crypto.dd.get_mt()

,Title,Description,Requires Paid Key,Sources
id,,,,
sply.total.iss,Total Issuance,The sum USD value of all new native units issu...,False,Coinmetrics
sply.circ,Circulating Supply,The circulating supply acknowledges that token...,False,
txn.fee.avg,Average Transaction Fees,The USD value of the mean fee per transaction ...,False,Coinmetrics
txn.tfr.val.adj.ntv,Adjusted Transaction Volume (Native Units),The sum of native units transferred between di...,False,Coinmetrics
txn.tfr.erc721.cnt,ERC-721 Transfer Count,The sum count of ERC-721 transfers in that int...,False,Coinmetrics
...,...,...,...,...
mcap.circ,Circulating Marketcap,The circulating marketcap is the price of the ...,False,"Messari,Kaiko"
cg.sply.circ,Circulating Supply (CoinGecko),The circulating supply acknowledges that token...,False,
real.vol,Real Volume,It is well known that many exchanges conduct w...,False,Kaiko


In [157]:
data = openbb.crypto.dd.mt('eth', timeseries_id='price', start_date = '2010-01-01', interval = '1w')[0]
openbb.qa.line(data = data["close"], log_y = True, title = "ETH/USD - Weekly")

These examples represent only a few methods for fetching historical price data.  Explore the contents of each module to find more!